### Выравнивание структуры папок и файлов (переменование, перемещение)

In [4]:
import os
from pathlib import Path
import shutil

# ---------------------------------------------------------
# БАЗОВЫЕ ПУТИ
# ---------------------------------------------------------
notebooks_dir = Path.cwd()
datasets_dir = notebooks_dir.parent / "datasets"

base_dir = datasets_dir / "raw" / "HWR200_custom"

print("Рабочая директория:", base_dir)


# ---------------------------------------------------------
# ФУНКЦИЯ ДЛЯ ПЕРЕИМЕНОВАНИЯ ТЕКСТОВЫХ ФАЙЛОВ
# ---------------------------------------------------------
def rename_text_files():
    print("=== Переименование текстовых файлов ===")

    # Перебираем три большие папки
    for range_folder in base_dir.iterdir():
        if not range_folder.is_dir():
            continue

        # Например "hwr200_0_19"
        range_name = range_folder.name

        # Внутри неё 0 или 20 или 50
        for subfolder in range_folder.iterdir():
            if not subfolder.is_dir():
                continue

            # Например "0"
            index_name = subfolder.name

            # Обрабатываем файлы reuseX.txt
            for i in range(13):
                old_name = subfolder / f"reuse{i}.txt"
                if not old_name.exists():
                    print("Пропущено (нет файла):", old_name)
                    continue

                new_name = subfolder / f"{range_name}_{index_name}_{i}.txt"

                print(f"{old_name.name} → {new_name.name}")
                old_name.rename(new_name)


# ---------------------------------------------------------
# ФУНКЦИЯ ДЛЯ ПЕРЕИМЕНОВАНИЯ ИЗОБРАЖЕНИЙ
# ---------------------------------------------------------
def rename_images():
    print("=== Переименование изображений ===")

    scan_map = {
        "Сканы": "",           # обычная версия
        "ФотоСветлое": "_l",   # light
        "ФотоТемное": "_d",    # dark
    }

    # Перебираем три большие папки ("hwr200_0_19", ...)
    for range_folder in base_dir.iterdir():
        if not range_folder.is_dir():
            continue

        range_name = range_folder.name  # пример: "hwr200_0_19"

        # Папки "0", "20", "50"
        for subfolder in range_folder.iterdir():
            if not subfolder.is_dir():
                continue

            index_name = subfolder.name  # пример: "0"

            # Ищем reuse0..reuse12
            for i in range(13):
                reuse_folder = subfolder / f"reuse{i}"
                if not reuse_folder.exists():
                    print("Нет папки:", reuse_folder)
                    continue

                # Теперь внутри три папки со сканами
                for scan_type, suffix in scan_map.items():
                    scan_path = reuse_folder / scan_type
                    if not scan_path.exists():
                        print("Нет подпапки:", scan_path)
                        continue

                    img_path = scan_path / "1.JPG"
                    if not img_path.exists():
                        print("Нет изображения:", img_path)
                        continue

                    # Генерация нового имени
                    new_file_name = f"{range_name}_{index_name}_{i}{suffix}.JPG"
                    new_path = scan_path / new_file_name

                    print(f"{img_path} → {new_path}")
                    img_path.rename(new_path)


# ---------------------------------------------------------
# ЗАПУСК ПЕРЕИМЕНОВАНИЯ
# ---------------------------------------------------------
rename_text_files()
rename_images()

print("\nГотово!")


Рабочая директория: c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom
=== Переименование текстовых файлов ===
Пропущено (нет файла): c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\reuse0.txt
Пропущено (нет файла): c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\reuse1.txt
Пропущено (нет файла): c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\reuse2.txt
Пропущено (нет файла): c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\reuse3.txt
Пропущено (нет файла): c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\reuse4.txt
Пропущено (нет файла): c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\reuse5.txt
Пропущено (нет файла): c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\reuse6.txt
Пропущено (н

### Переименование с перемещением изображений в родительскую папку

In [5]:
import os
import shutil

# Корень датасета (папка notebooks находится на одном уровне)
BASE_DIR = os.path.abspath("../datasets/raw/HWR200_custom")

# Верхнеуровневые папки
GROUPS = [
    ("hwr200_0_19", "0"),
    ("hwr200_20_39", "20"),
    ("hwr200_40_59", "50"),
]

# Подпапки изображений
IMAGE_TYPES = {
    "Сканы": "",        # базовый
    "ФотоСветлое": "_l",
    "ФотоТемное": "_d",
}

def rename_text_files(group_path, group_folder, number_folder):
    """
    Переименовать reuse{X}.txt → hwr200_A_B_C_X.txt
    """
    txt_folder = os.path.join(group_path, number_folder)
    prefix = f"{group_folder}_{number_folder}"

    for i in range(13):  # reuse0.txt … reuse12.txt
        old_name = os.path.join(txt_folder, f"reuse{i}.txt")
        if not os.path.isfile(old_name):
            continue  # уже переименован или отсутствует

        new_name = os.path.join(txt_folder, f"{prefix}_{i}.txt")
        os.rename(old_name, new_name)
        print(f"[TXT] {old_name} → {new_name}")


def move_and_rename_images(group_path, group_folder, number_folder):
    """
    Переместить и переименовать изображения:
    Сканы/1.JPG → reuseX/group_A_B_C_X.JPG
    ФотоСветлое/1.JPG → reuseX/group_A_B_C_X_l.JPG
    ФотоТемное/1.JPG → reuseX/group_A_B_C_X_d.JPG
    """
    base_number_path = os.path.join(group_path, number_folder)

    for reuse_id in range(13):
        reuse_folder = os.path.join(base_number_path, f"reuse{reuse_id}")
        if not os.path.isdir(reuse_folder):
            continue

        for folder_name, suffix in IMAGE_TYPES.items():
            src_image = os.path.join(reuse_folder, folder_name, "1.JPG")

            # Если файл источника нет — пропускаем
            if not os.path.isfile(src_image):
                continue

            new_filename = f"{group_folder}_{number_folder}_{reuse_id}{suffix}.JPG"
            dst_image = os.path.join(reuse_folder, new_filename)

            # Если файл уже переименован — пропустить
            if os.path.exists(dst_image):
                continue

            shutil.move(src_image, dst_image)
            print(f"[IMG] {src_image} → {dst_image}")


def process_all():
    for group_folder, number_folder in GROUPS:
        group_path = os.path.join(BASE_DIR, group_folder)

        print(f"\n==== Обработка группы {group_folder}/{number_folder} ====")

        # 1. Переименование текстовых файлов
        rename_text_files(group_path, group_folder, number_folder)

        # 2. Переименование и перенос изображений
        move_and_rename_images(group_path, group_folder, number_folder)


# Запуск
process_all()



==== Обработка группы hwr200_0_19/0 ====
[TXT] c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\reuse0.txt → c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\hwr200_0_19_0_0.txt
[TXT] c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\reuse1.txt → c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\hwr200_0_19_0_1.txt
[TXT] c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\reuse2.txt → c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\hwr200_0_19_0_2.txt
[TXT] c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\reuse3.txt → c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\hwr200_0_19_0_3.txt
[TXT] c:\Users\bleshchenko\YandexDisk\HandwrittenOCR\datasets\raw\HWR200_custom\hwr200_0_19\0\reus